## dependencies

In [1]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, window, unix_timestamp, max
from pyspark.sql.window import Window
from pyspark.sql import functions as F
import math
import time

from delta import *
from delta.tables import *
from pyspark.sql.functions import col, to_json, struct, lit, current_timestamp, expr, when, from_json, window
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    DoubleType,
    TimestampType,
    IntegerType,
)
import pandas as pd
import os
import uuid
import json

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


## schema definition

In [2]:
# Create a consistent warehouse directory path - use absolute path
WAREHOUSE_DIR = "/home/jovyan/spark-warehouse"

# Create the schema definition for NYC taxi data
def create_raw_taxi_schema():
    """
    Create the complete schema for NYC taxi data based on the DEBS Grand Challenge 2015 specification
    
    This schema includes all fields from the original dataset:
    - Basic identifiers (medallion, hack_license)
    - Time data (pickup_datetime, dropoff_datetime, trip_time_in_secs)
    - Trip information (trip_distance)
    - Location coordinates (pickup/dropoff longitude/latitude)
    - Payment information (payment_type, fare_amount, surcharge, mta_tax, tip_amount, tolls_amount)
    """
    return StructType([
        # Taxi and driver identifiers
        StructField("medallion", StringType(), True),         # Taxi vehicle identifier (md5sum)
        StructField("hack_license", StringType(), True),      # Taxi license identifier (md5sum)
        
        # Trip time information
        StructField("pickup_datetime", TimestampType(), True),   # Time of passenger pickup
        StructField("dropoff_datetime", TimestampType(), True),  # Time of passenger dropoff
        StructField("trip_time_in_secs", IntegerType(), True),   # Duration of the trip in seconds
        
        # Trip distance
        StructField("trip_distance", DoubleType(), True),     # Trip distance in miles
        
        # Pickup coordinates
        StructField("pickup_longitude", DoubleType(), True),  # Longitude coordinate of pickup
        StructField("pickup_latitude", DoubleType(), True),   # Latitude coordinate of pickup
        
        # Dropoff coordinates
        StructField("dropoff_longitude", DoubleType(), True), # Longitude coordinate of dropoff
        StructField("dropoff_latitude", DoubleType(), True),  # Latitude coordinate of dropoff
        
        # Payment information
        StructField("payment_type", StringType(), True),      # Payment method (credit card or cash)
        StructField("fare_amount", DoubleType(), True),       # Fare amount in dollars
        StructField("surcharge", DoubleType(), True),         # Surcharge in dollars
        StructField("mta_tax", DoubleType(), True),           # Tax in dollars
        StructField("tip_amount", DoubleType(), True),        # Tip in dollars
        StructField("tolls_amount", DoubleType(), True),      # Bridge and tunnel tolls in dollars
        
        # Additional fields that may be present
        StructField("total_amount", DoubleType(), True)       # Total amount paid (calculated field)
    ])

In [3]:
# Initialize Spark Session
def create_spark_session(app_name="FrequentRoutes"):
    """
    start spark session with kafka and delta support / memory config setup too
    """
    builder = SparkSession.builder.appName(app_name) \
        .config("spark.sql.session.timeZone", "UTC") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .config("spark.sql.warehouse.dir", WAREHOUSE_DIR) \
        .config("spark.sql.catalogImplementation", "hive") \
        .config("spark.driver.memory", "5g") \
        .config("spark.executor.memory", "4g") \
        .config("spark.memory.offHeap.enabled", "true") \
        .config("spark.memory.offHeap.size", "2g") \
        .config("spark.driver.maxResultSize", "2g") \
        .config("spark.sql.shuffle.partitions", "100") \
        .config("spark.default.parallelism", "100") \
        .config("spark.memory.fraction", "0.8") \
        .config("spark.sql.debug.maxToStringFields", 100) \
        .enableHiveSupport()
    
    # delta config
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    
    # do not flood logs
    spark.sparkContext.setLogLevel("WARN")
    
    # Print configs for debugging
    print(f"Warehouse directory: {spark.conf.get('spark.sql.warehouse.dir')}")
    print(f"Catalog implementation: {spark.conf.get('spark.sql.catalogImplementation')}")
    
    return spark

In [4]:
# Initialize the Spark session
spark = create_spark_session()

Warehouse directory: file:/home/jovyan/spark-warehouse
Catalog implementation: hive


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 46698)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 267, in poll
    if self.rfile in r and func():
                           ^^^^^^
  File "/usr/local/spark/python/pyspark/accumulators.py", line 271, in accum_updates
    num_updates =

In [5]:
raw_taxi_schema = create_raw_taxi_schema()

In [6]:
# Step 1: Define paths
CLEAN_TABLE_NAME = "clean_taxi_data"
FREQ_TABLE_NAME = "freq_taxi_data"

CLEAN_OUTPUT_PATH = os.path.join(WAREHOUSE_DIR, CLEAN_TABLE_NAME)
FREQ_OUTPUT_PATH = os.path.join(WAREHOUSE_DIR, FREQ_TABLE_NAME)

CHECKPOINT_DIR = os.path.join(WAREHOUSE_DIR, "streaming/checkpoints")
CLEAN_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "clean_taxi_data")

FREQ_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "frequent_routes")
FREQ_ROUTES_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "frequent_routes")

In [7]:
# Function to create a table once streaming data is available
def create_table_if_exists(output_path, table_name):
    """
    Check if data exists in the given path and create a table pointing to it
    """
    data_exists = False
    for _i in range(30):  # Longer timeout to 30 seconds
        try:
            time.sleep(120)
            if os.path.exists(output_path):
                files = os.listdir(output_path)
                for _f in files:
                    if ".parquet" in _f:
                        if os.path.exists(f"{output_path}/_delta_log") and len(os.listdir(f"{output_path}/_delta_log")) > 0:
                            print(f"Data exists in {output_path}")
                            data_exists = True
                            break
            if data_exists:
                # Create external table with explicit location
                spark.sql(f"DROP TABLE IF EXISTS {table_name}")
                spark.sql(f"CREATE TABLE {table_name} USING DELTA LOCATION '{output_path}'")
                print(f"Created table {table_name} using data at {output_path}")
                break
        except Exception as e:
            print(f"Waiting for data: {e}")
            pass
    
    if not data_exists:
        print(f"WARNING: No data found in {output_path} after waiting. Table may not be created.")


In [8]:
# Define constants
CELL_SIZE = 500  # size of grid cell in meters
ORIGIN_LAT = 41.474937  # Latitude of cell 1.1
ORIGIN_LON = -74.913585  # Longitude of cell 1.1
MAX_CELL = 300  # grid expands 150km with 500m cells

In [9]:
# Helper functions to convert lat/lon to cell coordinates
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Function to compute grid cell ID
def get_cell_id(lat, lon):
    base_lat, base_lon = 41.474937, -74.913585  # NYC reference point
    cell_size = 0.0045  # Approx. 500m in degrees

    cell_x = int((lon - base_lon) / cell_size) + 1
    cell_y = int((base_lat - lat) / cell_size) + 1
    return f"{cell_x}.{cell_y}"

# Register UDFs
spark.udf.register("get_cell_id", get_cell_id, StringType())


<function __main__.get_cell_id(lat, lon)>

In [10]:
df = spark.readStream.format("delta").load(CLEAN_OUTPUT_PATH)

In [11]:
# Part 1: top 10 in 30 minutes window. data is already sorted.
def query1_part1(clean_taxi_data):
    # Convert coordinates to grid cells
    routes_df = df.withColumn(
        "start_cell", 
        F.expr("get_cell_id(pickup_latitude, pickup_longitude)")
    ).withColumn(
        "end_cell", 
        F.expr("get_cell_id(dropoff_latitude, dropoff_longitude)")
    )    
    
    # 30 minutes sliding window
    windowed_routes = routes_df.withWatermark("dropoff_datetime", "30 minutes") \
        .groupBy(
            F.window("dropoff_datetime", "30 minutes"),
            "start_cell", 
            "end_cell"
        ) \
        .count() \
        .withColumnRenamed("count", "Number_of_Rides")
    
    # top 10 routes from stream
    top_routes = windowed_routes \
        .select("window", "start_cell", "end_cell", "Number_of_Rides") \
        .orderBy(col("Number_of_Rides").desc()) \
        # .limit(10)
    return top_routes


In [12]:
batch_df = spark.read.format("delta").load(CLEAN_OUTPUT_PATH)
batch_df.select("dropoff_datetime").orderBy("dropoff_datetime", ascending=False).limit(5).show(truncate=False)

+-------------------+
|dropoff_datetime   |
+-------------------+
|2014-01-01 00:36:00|
|2014-01-01 00:27:13|
|2014-01-01 00:24:00|
|2014-01-01 00:23:00|
|2014-01-01 00:18:00|
+-------------------+



In [13]:
batch_df.head(10)

[Row(kafka_key=None, kafka_timestamp=datetime.datetime(2025, 3, 28, 17, 45, 9, 653000), medallion='3567E8B49FEBFCBB587F1864D723D5C8', hack_license='430B8022563CDE1D51D44786DFD8D6CB', pickup_datetime=datetime.datetime(2013, 10, 19, 20, 37, 8), dropoff_datetime=datetime.datetime(2013, 10, 19, 20, 57, 20), trip_time_in_secs=1211, trip_distance=2.0, pickup_longitude=-73.982338, pickup_latitude=40.731693, dropoff_longitude=-73.988419, dropoff_latitude=40.750069, payment_type='CRD', fare_amount=14.0, surcharge=0.5, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, total_amount=15.0, start_cell=None, end_cell=None, Number_of_Rides=None),
 Row(kafka_key=None, kafka_timestamp=datetime.datetime(2025, 3, 28, 17, 45, 9, 653000), medallion='B620FF1CD0B80A120894F7A1D376582D', hack_license='76AC51FFFF22C9A6416560A8F70E2CB0', pickup_datetime=datetime.datetime(2013, 4, 23, 15, 36), dropoff_datetime=datetime.datetime(2013, 4, 23, 15, 48), trip_time_in_secs=696, trip_distance=1.5, pickup_longitude=-73.99259

In [14]:
batch_df.select("dropoff_datetime").orderBy("dropoff_datetime").limit(5).show(truncate=False)

+-------------------+
|dropoff_datetime   |
+-------------------+
|2013-01-01 00:03:00|
|2013-01-01 00:05:49|
|2013-01-01 00:06:00|
|2013-01-01 00:06:46|
|2013-01-01 00:06:54|
+-------------------+



In [15]:
batch_df.columns

['kafka_key',
 'kafka_timestamp',
 'medallion',
 'hack_license',
 'pickup_datetime',
 'dropoff_datetime',
 'trip_time_in_secs',
 'trip_distance',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'payment_type',
 'fare_amount',
 'surcharge',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'total_amount',
 'start_cell',
 'end_cell',
 'Number_of_Rides']

In [16]:
batch_df.count()

6871525

In [17]:
null_longitude_count = batch_df.filter(F.col("pickup_longitude").isNull()).count()


In [18]:
null_longitude_count

0

In [19]:
batch_df.filter(F.col("pickup_latitude").isNull()).count()


0

In [20]:
nan_longitude_count = batch_df.filter(F.isnan("pickup_longitude")).count()
nan_latitude_count = batch_df.filter(F.isnan("pickup_latitude")).count()
nan_longitude_count, nan_latitude_count

(0, 0)

In [21]:
time_range = batch_df.agg(
    F.min("dropoff_datetime").alias("earliest_dropoff"),
    F.max("dropoff_datetime").alias("latest_dropoff")
).collect()[0]
    
print(f"\nData spans from {time_range['earliest_dropoff']} to {time_range['latest_dropoff']}")
    


Data spans from 2013-01-01 00:03:00 to 2014-01-01 00:36:00


In [22]:
clean_df = df.withColumn(
    "pickup_datetime", F.to_timestamp("pickup_datetime")
).withColumn(
    "dropoff_datetime", F.to_timestamp("dropoff_datetime")
)
    
# Run Query 1, Part 1
part1_results = query1_part1(clean_df)

query1_part1_stream = (part1_results.writeStream
    .outputMode("complete")
    .format("delta")
    .queryName("frequent_routes")
    .trigger(processingTime="3 second")
    .option("checkpointLocation", FREQ_CHECKPOINT_PATH)
    .start(FREQ_OUTPUT_PATH)
)

In [24]:
query1_part1_stream.status

{'message': 'Waiting for next trigger',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [ ]:
create_table_if_exists(FREQ_OUTPUT_PATH, FREQ_TABLE_NAME)

In [25]:
freq_batch_df = spark.read.format("delta").load(FREQ_OUTPUT_PATH)


In [26]:
freq_batch_df.count()

#pandas_df = freq_batch_df.toPandas()

# Print the Pandas DataFrame
#print(pandas_df)

6653605

In [27]:
latest_window = freq_batch_df \
    .select("window") \
    .orderBy(col("window.end").desc()) \
    .limit(1) \
    .collect()[0][0]

In [43]:
max_rides_window = freq_batch_df.orderBy(col("window.end").desc(), col("Number_Of_Rides").desc()).limit(1).collect()[0][0]

In [44]:
window_start, window_end = max_rides_window["start"], max_rides_window["end"]

In [45]:
window_start, window_end

(datetime.datetime(2014, 1, 1, 0, 30), datetime.datetime(2014, 1, 1, 1, 0))

In [28]:
latest_time = freq_batch_df.agg(F.max("window.end")).collect()[0][0]


In [29]:
latest_time

datetime.datetime(2014, 1, 1, 1, 0)

In [30]:
thirty_min_ago = latest_time - pd.Timedelta(minutes=30)

top_routes = freq_batch_df \
    .filter(col("window.end") >= thirty_min_ago) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        "start_cell", 
        "end_cell", 
        "Number_of_Rides"
    ) \
    .orderBy(col("Number_of_Rides").desc()) \
    .limit(10)

In [46]:
thirty_min_ago = window_end - pd.Timedelta(minutes=30)

top_routes_max_window = freq_batch_df \
    .filter(col("window.end") >= thirty_min_ago) \
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        "start_cell", 
        "end_cell", 
        "Number_of_Rides"
    ) \
    .orderBy(col("Number_of_Rides").desc()) \
    .limit(10)

In [55]:
df = freq_batch_df.withColumn("window_start", F.col("window.start")) \
       .withColumn("window_end", F.col("window.end"))

# Find top windows based on ride count
window_counts = df.groupBy("window_start", "window_end") \
    .agg(F.sum("Number_of_Rides").alias("total_rides")) \
    .orderBy(F.desc("total_rides"))

# Get the top window (or windows if there's a tie)
top_window = window_counts.first()
top_window_rides = top_window["total_rides"]
top_windows = window_counts.filter(F.col("total_rides") == top_window_rides)

# Get the top 10 routes for each top window
windowSpec = Window.partitionBy("window_start", "window_end").orderBy(F.desc("Number_of_Rides"))
routes_with_rank = df.join(
    top_windows, 
    ["window_start", "window_end"]
).withColumn("rank", F.rank().over(windowSpec))

# Get top 10 routes per window
top_routes = routes_with_rank.filter(F.col("rank") <= 10)

# If there are fewer than 10 routes for a window, we need to pad with nulls
# First, create a sequence from 1 to 10
rank_df = spark.range(1, 11).withColumnRenamed("id", "rank")

# Create a cross join between top windows and ranks
window_rank_template = top_windows.crossJoin(rank_df)

# Do a left outer join to fill missing ranks with nulls
final_result = window_rank_template.join(
    top_routes,
    ["window_start", "window_end", "rank"],
    "left_outer"
).orderBy("window_start", "window_end", "rank")

# Select only the columns we need for the final output
final_result = final_result.select(
    "window_start",
    "window_end",
    "rank",
    "start_cell",
    "end_cell",
    "Number_of_Rides"
)

# Display results
print("Top Windows by Ride Count:")
top_windows.show()

print("\nTop 10 Routes per Top Window (null for missing ranks):")
final_result.show(20)  

Top Windows by Ride Count:
+-------------------+-------------------+-----------+
|       window_start|         window_end|total_rides|
+-------------------+-------------------+-----------+
|2013-11-03 01:30:00|2013-11-03 02:00:00|       1033|
+-------------------+-------------------+-----------+


Top 10 Routes per Top Window (null for missing ranks):
+-------------------+-------------------+----+----------+----------+---------------+
|       window_start|         window_end|rank|start_cell|  end_cell|Number_of_Rides|
+-------------------+-------------------+----+----------+----------+---------------+
|2013-11-03 01:30:00|2013-11-03 02:00:00|   1|16648.9217|16648.9217|              6|
|2013-11-03 01:30:00|2013-11-03 02:00:00|   2|   206.165|   209.162|              3|
|2013-11-03 01:30:00|2013-11-03 02:00:00|   2|   207.162|   208.166|              3|
|2013-11-03 01:30:00|2013-11-03 02:00:00|   2|   206.165|   208.163|              3|
|2013-11-03 01:30:00|2013-11-03 02:00:00|   3|     

In [47]:
top_routes_max_window.head(10)

[Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='211.160', end_cell='210.158', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='252.185', end_cell='210.177', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='205.161', end_cell='216.144', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='202.168', end_cell='209.153', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='211.159', end_cell='212.155', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='207.157', end_cell='214.156', Number_of_Rides=1),
 Row

In [31]:
top_routes.head(10)


[Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='211.160', end_cell='210.158', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='252.185', end_cell='210.177', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='205.161', end_cell='216.144', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='202.168', end_cell='209.153', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='211.159', end_cell='212.155', Number_of_Rides=1),
 Row(window_start=datetime.datetime(2014, 1, 1, 0, 0), window_end=datetime.datetime(2014, 1, 1, 0, 30), start_cell='207.157', end_cell='214.156', Number_of_Rides=1),
 Row

In [32]:
pandas_df = top_routes.toPandas()

# Print the Pandas DataFrame
pandas_df

,window_start,window_end,start_cell,end_cell,Number_of_Rides
0,2014-01-01,2014-01-01 00:30:00,211.160,210.158,1
1,2014-01-01,2014-01-01 00:30:00,252.185,210.177,1
2,2014-01-01,2014-01-01 00:30:00,205.161,216.144,1
3,2014-01-01,2014-01-01 00:30:00,202.168,209.153,1
4,2014-01-01,2014-01-01 00:30:00,211.159,212.155,1
5,2014-01-01,2014-01-01 00:30:00,207.157,214.156,1
6,2014-01-01,2014-01-01 00:30:00,207.158,205.162,1
7,2014-01-01,2014-01-01 00:30:00,207.178,213.179,1
8,2014-01-01,2014-01-01 00:30:00,207.156,214.153,1
9,2014-01-01,2014-01-01 00:30:00,216.155,233.137,1


In [48]:
pandas_max_window_df = top_routes_max_window.toPandas()


In [49]:
pandas_max_window_df.head(10)

,window_start,window_end,start_cell,end_cell,Number_of_Rides
0,2014-01-01,2014-01-01 00:30:00,211.160,210.158,1
1,2014-01-01,2014-01-01 00:30:00,252.185,210.177,1
2,2014-01-01,2014-01-01 00:30:00,205.161,216.144,1
3,2014-01-01,2014-01-01 00:30:00,202.168,209.153,1
4,2014-01-01,2014-01-01 00:30:00,211.159,212.155,1
5,2014-01-01,2014-01-01 00:30:00,207.157,214.156,1
6,2014-01-01,2014-01-01 00:30:00,207.158,205.162,1
7,2014-01-01,2014-01-01 00:30:00,207.178,213.179,1
8,2014-01-01,2014-01-01 00:30:00,207.156,214.153,1
9,2014-01-01,2014-01-01 00:30:00,216.155,233.137,1


```bash
big_data_project_pyspark  | Batch: 0
big_data_project_pyspark  | -------------------------------------------
big_data_project_pyspark  | +----------+--------+---------------+
big_data_project_pyspark  | |start_cell|end_cell|Number_of_Rides|
big_data_project_pyspark  | +----------+--------+---------------+
big_data_project_pyspark  | |   156.160| 157.159|              7|
big_data_project_pyspark  | |   156.160| 157.159|              7|
big_data_project_pyspark  | |   156.160| 157.159|              7|
big_data_project_pyspark  | |   157.159| 154.160|              7|
big_data_project_pyspark  | |   157.159| 154.160|              7|
big_data_project_pyspark  | |   158.160| 159.159|              7|
big_data_project_pyspark  | |   156.160| 157.159|              6|
big_data_project_pyspark  | |   156.160| 158.160|              6|
big_data_project_pyspark  | |   158.160| 159.159|              6|
big_data_project_pyspark  | |   156.160| 157.159|              6|
big_data_project_pyspark  | +----------+--------+---------------+
```

In [ ]:
# Show results
# XXX: check this :'c top_routes.show()

In [ ]:
# Run the batch processing
part2_results = query1_part2_batch(df)


In [ ]:
part2_results.show(truncate=False)